In [ ]:
!pip install pysus
!pip install geopandas

Para melhor compreensão do dataset visite o
[dicionario de dados](https://pcdas.icict.fiocruz.br/conjunto-de-dados/sistema-de-informacoes-de-mortalidade-sim/dicionario-de-variaveis/)

In [ ]:
from pysus.online_data import SIM, parquets_to_dataframe
from pysus.online_data.SIM import download

from pysus.preprocessing.decoders import decodifica_idade_SIM, translate_variables_SIM
import pandas as pd
import numpy as np

In [ ]:

estados = ['RS']

dataFrame = pd.DataFrame()

ano = 2010
while ano <= 2021:
    file_path = download('RS', ano)
    dataFrame_anual = pd.read_parquet(file_path)

    if dataFrame_anual.shape[0] > 50000:  # Limit data to 5000 rows
        dataFrame_anual = dataFrame_anual.sample(50000, random_state=np.random.randint(1, 10000))

    # Add extra useful columns
    dataFrame_anual['UF'] = 'RS'
    dataFrame_anual['YEAR'] = ano
    dataFrame_anual['DTNASC'] = dataFrame_anual['DTNASC'].str.strip().replace('', np.nan)
    dataFrame_anual['DTOBITO'] = dataFrame_anual['DTOBITO'].str.strip().replace('', np.nan)

    print("Data from " + str(ano) + " for " + str('RS') + " downloaded and processed! " + str(dataFrame_anual.shape[0]) + " rows")

    dataFrame = pd.concat([dataFrame, dataFrame_anual], axis=0)  # Concatenate DataFrames
    ano += 1

# Remove missing values and duplicates
dataFrame.drop_duplicates()

print(dataFrame.info())

Data from 2010 for RS downloaded and processed! 9319 rows
Data from 2011 for RS downloaded and processed! 50000 rows
Data from 2012 for RS downloaded and processed! 50000 rows
Data from 2013 for RS downloaded and processed! 50000 rows
Data from 2014 for RS downloaded and processed! 50000 rows
Data from 2015 for RS downloaded and processed! 50000 rows
Data from 2016 for RS downloaded and processed! 50000 rows
Data from 2017 for RS downloaded and processed! 50000 rows
Data from 2018 for RS downloaded and processed! 50000 rows
Data from 2019 for RS downloaded and processed! 50000 rows
Data from 2020 for RS downloaded and processed! 50000 rows
Data from 2021 for RS downloaded and processed! 50000 rows
<class 'pandas.core.frame.DataFrame'>
Int64Index: 559319 entries, 0 to 16148
Data columns (total 98 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   CONTADOR    559319 non-null  object
 1   ORIGEM      559319 non-null  object
 2   TIPOBITO    55

In [ ]:
# Mantendo apenas colunas desejadas
columns = ['DTOBITO', 'CODMUNNATU', 'IDADE', 'SEXO', 'ASSISTMED', 'EXAME', 'CIRURGIA']
dataFrame = dataFrame.drop(columns=list(set(dataFrame.columns) - set(columns)))
print(dataFrame.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559319 entries, 0 to 16148
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   DTOBITO     559319 non-null  object
 1   IDADE       559319 non-null  object
 2   SEXO        559319 non-null  object
 3   ASSISTMED   559319 non-null  object
 4   EXAME       559319 non-null  object
 5   CIRURGIA    559319 non-null  object
 6   CODMUNNATU  400000 non-null  object
dtypes: object(7)
memory usage: 34.1+ MB
None


In [ ]:
# Transformando os dados
dataFrame['IDADE'] = dataFrame['IDADE'].str.strip().astype(int) - 400
dataFrame['IDADE'] = dataFrame['IDADE'].where(dataFrame['IDADE'] >= 0, 0)
dataFrame['DTOBITO'] = pd.to_datetime(dataFrame['DTOBITO'], format='%d%m%Y')
dataFrame

,DTOBITO,IDADE,SEXO,ASSISTMED,EXAME,CIRURGIA,CODMUNNATU
0,2010-08-14,95,2,,2,,NaN
1,2010-02-26,73,1,,,,NaN
2,2010-07-22,54,1,,2,2,NaN
3,2010-03-02,19,1,,2,2,NaN
4,2010-03-02,11,1,,2,2,NaN
...,...,...,...,...,...,...,...
92513,2021-04-29,91,2,,,,430450
10843,2021-08-25,85,1,1,,,431490
96530,2021-03-17,91,2,1,,,430420
53763,2021-06-21,52,2,1,,,430610


In [ ]:
# Conversão e donwload
dataFrame.to_csv('SIM_dataframe.csv')

In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive')

import shutil

# Define the file path in Colab
colab_file_path = './sim_data_cleaned_2021.csv'  # Replace with your file's actual path

# Define the destination file path in Google Drive
drive_file_path = '/content/drive/My Drive/sim_data_cleaned_2021.csv'

# Use shutil to copy the file to Google Drive
shutil.copy2(colab_file_path, drive_file_path)

# Print a message to confirm the upload
print(f"File uploaded to Google Drive: {drive_file_path}")
'''

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n\nimport shutil\n\n# Define the file path in Colab\ncolab_file_path = \'./sim_data_cleaned_2021.csv\'  # Replace with your file\'s actual path\n\n# Define the destination file path in Google Drive\ndrive_file_path = \'/content/drive/My Drive/sim_data_cleaned_2021.csv\'\n\n# Use shutil to copy the file to Google Drive\nshutil.copy2(colab_file_path, drive_file_path)\n\n# Print a message to confirm the upload\nprint(f"File uploaded to Google Drive: {drive_file_path}")\n'

pysus SIA data